In [1]:
import os 
current_dir = os.getcwd()
print(current_dir)
os.chdir("../")

d:\interview\End-to-End-DL\research


In [2]:
from cnnClassifier.utils import read_yaml , create_directories
from cnnClassifier.constants import  *
from dataclasses import dataclass
from pathlib import Path
import os
from cnnClassifier import logging
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path:Path
    update_base_model:Path
    training_data: Path
    params_epoch: int
    params_batch_size : int
    params_is_augmented : bool
    params_image_size: list

class ConfigurationManager:
    def __init__(self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH):
        self.config_file = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config_file.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbackConfig:
        config = self.config_file.prepare_callbacks
        
        model_ckp = os.path.dirname(config.checkpoint_model_filepath)


        create_directories([Path(model_ckp) , Path(config.tensorboard_root_log_dir)])
        
        prepare_callback_config = PrepareCallbackConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        return prepare_callback_config
    
    def get_training_configuration(self) -> TrainingConfig:
        training = self.config_file.training
        prepare_base_model = self.config_file.prepare_base_model
        params = self.params
        training_data  = os.path.join(self.config_file.data_ingestion.unzip_dir ,"PetImages" )
        training_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            update_base_model = Path(prepare_base_model.update_base_model_path),
            training_data = Path(training_data),
            params_batch_size = int(params.BATCH_SIZE),
            params_epoch = int(params.EPOCHS),
            params_is_augmented = bool(params.AUGMENTATION),
            params_image_size = list(params.IMAGE_SIZE)
        )
        return training_config

In [3]:
import tensorflow as tf
import time 
class PrepareCallback:  
    def __init__(self, config:PrepareCallbackConfig):
        self.config = config
    
    @property
    def _create_tensorboard_callback(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_log_dir = os.path.join(self.config.tensorboard_root_log_dir, f"tb_log_dir_{timestamp}")

        return tf.keras.callbacks.TensorBoard(log_dir=tb_log_dir)
    
    @property
    def _create_ckpt_callback(self):
        return  tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_filepath,
            save_best_only=True
        )

    
    def get_tb_ckpt_callback(self):
        return [
            self._create_tensorboard_callback,
            self._create_ckpt_callback
        ]


[2024-01-30 01:44:36,657: INFO: utils: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.]
[2024-01-30 01:44:36,659: INFO: utils: NumExpr defaulting to 8 threads.]


In [9]:
class Training:
    def __init__(self , config:TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.update_base_model
        )
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmented:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epoch,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [10]:

try:
    config_manager = ConfigurationManager()
    callback_config = config_manager.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config = callback_config)
    callback = prepare_callbacks.get_tb_ckpt_callback()
    training_config = config_manager.get_training_configuration()
    training = Training(config = training_config)
    training.get_base_model()
    training.train_valid_generator()
    
    training.train(callback_list=callback)
    
except Exception as e:
    print(e)
    logging(e)
    

[2024-01-30 01:47:46,893: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-30 01:47:46,902: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-30 01:47:46,906: INFO: common: created directory at: artifacts]
[2024-01-30 01:47:46,907: INFO: common: created directory at: artifacts\prepare_callbacks]
[2024-01-30 01:47:46,907: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_logs]
Found 78 images belonging to 2 classes.
Found 318 images belonging to 2 classes.
19/19 [==============================] - 39s 1s/step - loss: 9.2573 - accuracy: 0.5331 - val_loss: 2.2470 - val_accuracy: 0.7188
